# Applied Data Science Capstone - Neighborhoods in Toronto: explore and clustering

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#!conda install -c conda-forge geocoder 
!pip install geocoder
import geocoder # import geocoder

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests
print('libs imported!')

I'll use pandas to read the wikipedia table using the method read_html and extracting the first table with css class 'wikitable'. (es: pandas.read_html(url, attrs={'class': 'wikitable'}))   

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url, attrs={'class': 'wikitable'})   
df = pd.concat(data)

#3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

#3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#df = df[df['Borough'] != 'Not assigned']
df.drop(df[df['Borough']=="Not assigned"].index, axis=0, inplace=True)

#3.3  More than one neighborhood can exist in one postal code area. 
#     For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#     Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = pd.DataFrame(df.groupby('PostalCode').agg(lambda x:', '.join(x.unique())))
df.reset_index(inplace=True)

#3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#    So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [3]:
df.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
def geocoder_google(postal_code, city= 'Toronto', country='Ontario'):
    
    # initialize your variable to None
    lat_lng_coords = None
    i = 0
    
    # loop until you get the coordinates
    while(lat_lng_coords is None and i < 3):
        address = '{}, {}, {}'.format(postal_code, city, country)
        g = geocoder.google(address, key=GOOGLE_API_KEY)
        lat_lng_coords = g.latlng
        i = i + 1
        #print(postal_code, lat_lng_coords)
    return lat_lng_coords

In [6]:
df['Coords'] = df.apply(lambda x: geocoder_google(x['PostalCode']), axis=1)
df['Latitude'] = df.apply(lambda x: x['Coords'][0], axis=1)
df['Longitude'] = df.apply(lambda x: x['Coords'][1], axis=1)
df.drop('Coords', axis=1, inplace=True)

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
df.shape

(103, 5)

In [8]:
df.to_csv('toronto.csv', index=False)

In [9]:
df1 = pd.read_csv('toronto.csv')

df1.head()

print(df.shape, df1.shape)

(103, 5) (103, 5)


## Explore the neighborhoods in Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. Just make sure:
   * to add enough Markdown cells to explain what you decided to do and to report any observations you make.
   * to generate maps to visualize your neighborhoods and how they cluster together. 

In [10]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: PHHF1GSKBJUGYWOKFIMGKJUFOIXBWEDQGYRNSGVSFGGH1BDS
CLIENT_SECRET:KH1DYCIEQMOANMNGZQJQN1XG0IDLWJT4O0NDX3ZMGZS03G2N


In [11]:
#filter the Borough that contain the word Toronto
df_toronto = df[df['Borough'].str.contains('Toronto')].sort_values('Borough')

print(df_toronto.shape)
df_toronto.head(10)

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [12]:
# explore first n venues near postal codes coords
def explore_venues_in_toronto(postal_codes, borough, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postal_code, borough, lat, lng in zip(postal_codes, borough, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #print(results[0]['venue'])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            borough,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                             'Borough',
                             'PostalCode Latitude',
                             'PostalCode Longitude',
                             'Venue',
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [13]:
df_venues = explore_venues_in_toronto(df_toronto['PostalCode'], df_toronto['Borough'], df_toronto['Latitude'], df_toronto['Longitude'])

print(df_venues.shape)
df_venues.head()

(1714, 8)


,PostalCode,Borough,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5R,Central Toronto,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
1,M5R,Central Toronto,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,M5R,Central Toronto,43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
3,M5R,Central Toronto,43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant
4,M5R,Central Toronto,43.67271,-79.405678,Fet Zun,43.675147,-79.406346,Middle Eastern Restaurant


Let's check how many venues were returned for each PostalCode

In [14]:
print('number of distinct Borough:', len(df_venues['Borough'].unique()))

print()
print('Let''s check how many venues were returned for each PostalCode')
df_venues.groupby('PostalCode').count()

number of distinct Borough: 4

Lets check how many venues were returned for each PostalCode


,Borough,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,,
M4E,5,5,5,5,5,5,5
M4K,42,42,42,42,42,42,42
M4L,20,20,20,20,20,20,20
M4M,42,42,42,42,42,42,42
M4N,4,4,4,4,4,4,4
M4P,7,7,7,7,7,7,7
M4R,22,22,22,22,22,22,22
M4S,33,33,33,33,33,33,33
M4T,1,1,1,1,1,1,1


In [15]:
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 236 uniques categories.


## Analyze neighborhoods

In [16]:
# first two coloumns of the new dataframe will be Postal Code and Borough
df_analysis = df_venues[['PostalCode', 'Borough']]

# generate dataframe of dummies venue categories 
df_dummies_venues_categories = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="") 

# concatenate the dummy venues categories dataframe to the initial dataframe 
df_analysis = pd.concat([df_analysis, df_dummies_venues_categories], sort=False, axis=1)

print('df_analysis shape:', df_analysis.shape)
df_analysis.head()

df_analysis shape: (1714, 238)


,PostalCode,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5R,Central Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5R,Central Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5R,Central Toronto,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,M5R,Central Toronto,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,M5R,Central Toronto,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by Postal Code and by taking the mean of the frequency of occurrence of each category

In [17]:
df_means_venues_categories = df_analysis.groupby('PostalCode').mean().reset_index()

print('df_means_venues_categories shape:', df_means_venues_categories.shape)
df_means_venues_categories.head()

df_means_venues_categories shape: (39, 237)


,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.20000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,...,0.0,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.02381
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.02381
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [18]:
def get_top_categories_venues(row, num_top_venues):
    row_categories = row.iloc[1:].sort_values(ascending=False).index.values[0:num_top_venues]
    
    return row_categories

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_clustering_venues = pd.DataFrame(columns=columns)
df_clustering_venues['PostalCode'] = df_means_venues_categories['PostalCode']

for ind in np.arange(df_means_venues_categories.shape[0]):
    df_clustering_venues.iloc[ind, 1:] = get_top_categories_venues(df_means_venues_categories.iloc[ind, :], num_top_venues)

print('df_clustering_venues shape:', df_clustering_venues.shape)    

print()
print('df_clustering_venues top 5')
df_clustering_venues.head()

df_clustering_venues shape: (39, 11)

df_clustering_venues top 5


,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Neighborhood,Other Great Outdoors,Health Food Store,Trail,Pub,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Brewery,Bubble Tea Shop,Dessert Shop
2,M4L,Park,Sandwich Place,Gym,Movie Theater,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,Café,Coffee Shop,Brewery,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Convenience Store,Sandwich Place
4,M4N,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Clustering

Run *k*-means to cluster the postal codes into 5 clusters.

In [20]:
# set number of clusters
kclusters = 5

# drop the postalCode column and run k-means clustering on dataframe with the venue category frequencies
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_means_venues_categories.drop('PostalCode', 1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [21]:
# add clustering labels
df_clustering_venues.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each postal code
df_merged = df_toronto.join(df_clustering_venues.set_index('PostalCode'), on='PostalCode')

print('df_merged shape:', df_merged.shape)
df_merged.head() # check the last columns!

df_merged shape: (39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,Pharmacy,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,Liquor Store,History Museum
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Trail,Jewelry Store,Park,Sushi Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym,Hotel,Park,Breakfast Spot,Department Store,Sandwich Place,Food & Drink Shop,Distribution Center,Dog Run,Donut Shop
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Dessert Shop,Restaurant,Mexican Restaurant,Sporting Goods Shop,Chinese Restaurant,Fast Food Restaurant,Spa


Finally, let's visualize the resulting clusters

In [22]:
# create map
location=[df_merged.mean()['Latitude'], df_merged.mean()['Longitude']]
map_clusters = folium.Map(location=location, zoom_start=11).add_to(folium.Figure(width=750, height=400))

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['PostalCode'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examin Clusters

Based on the defining categories, we will assign a label to each cluster.

In [23]:
# get cluster
def get_cluster(df, cluster_label):
    return df.loc[df['Cluster Labels'] == cluster_label, df.columns[[1] + list(range(5, df.shape[1]))]]

Cluster 1

In [24]:
get_cluster(df_merged, 0)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,0,Trail,Jewelry Store,Park,Sushi Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
50,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 2

In [25]:
get_cluster(df_merged, 1)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Pharmacy,Burger Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,Liquor Store,History Museum
45,Central Toronto,1,Gym,Hotel,Park,Breakfast Spot,Department Store,Sandwich Place,Food & Drink Shop,Distribution Center,Dog Run,Donut Shop
46,Central Toronto,1,Clothing Store,Coffee Shop,Yoga Studio,Dessert Shop,Restaurant,Mexican Restaurant,Sporting Goods Shop,Chinese Restaurant,Fast Food Restaurant,Spa
47,Central Toronto,1,Dessert Shop,Pizza Place,Sandwich Place,Gym,Sushi Restaurant,Italian Restaurant,Café,Coffee Shop,Brewery,Gas Station
49,Central Toronto,1,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Sports Bar,Supermarket
59,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Italian Restaurant,Hotel,Restaurant,Brewery,Fried Chicken Joint,Scenic Lookout,History Museum
66,Downtown Toronto,1,Café,Restaurant,Bakery,Bar,Bookstore,Japanese Restaurant,Dessert Shop,Pub,Noodle House,Nightclub
61,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Gym,Seafood Restaurant,American Restaurant,Italian Restaurant,Gastropub,Deli / Bodega
60,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Japanese Restaurant,Gastropub,Bar,Bakery,Seafood Restaurant
85,Downtown Toronto,1,Coffee Shop,Gym,Park,Burger Joint,Diner,Portuguese Restaurant,Music Venue,Mexican Restaurant,Juice Bar,Japanese Restaurant


Cluster 3

In [26]:
get_cluster(df_merged, 2)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,2,Playground,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 4

In [27]:
get_cluster(df_merged, 3)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Garden,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 5

In [28]:
get_cluster(df_merged, 4)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Photography Studio,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
